In [3]:
import pandas as pd
import function
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential
from keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import SGD
from keras.losses import SparseCategoricalCrossentropy

In [2]:
train_vec=pd.read_csv('./gc_data/train_vec.txt',sep=' ',header=None)
dev_vec=pd.read_csv('./gc_data/dev_vec.txt',sep=' ',header=None)
test_vec=pd.read_csv('./gc_data/test_vec.txt',sep=' ',header=None)
train_label=pd.read_csv('./gc_data/train_label.txt',names=['label'])
dev_label=pd.read_csv('./gc_data/dev_label.txt',names=['label'])
test_label=pd.read_csv('./gc_data/test_label.txt',names=['label'])

In [3]:
train_label=function.transform_label(train_label)
dev_label=function.transform_label(dev_label)
test_label=function.transform_label(test_label)

In [4]:
train_X=np.array(train_vec)
train_Y=np.array(list(train_label['label'].astype(int)))
dev_X=np.array(dev_vec)
dev_Y=np.array(list(dev_label['label'].astype(int)))
test_X=test_vec
test_Y=test_label['label'].astype(int)

In [ ]:
def MLP(optimizer=SGD(),loss=SparseCategoricalCrossentropy(),metrics=['accuracy']):
    model=Sequential([
        Dense(256,activation='relu',input_shape=(768,)),
        Dense(256,activation='relu'),
        Dense(128,activation='relu'),
        Dense(64,activation='relu'),
        Dropout(0.2),
        Dense(18,activation='softmax')
    ])
    model.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return model

model=MLP('adam')
history=model.fit(train_X,train_Y,batch_size=64,epochs=100,validation_data=(dev_X,dev_Y))
model.save('MLP.h5')
result=model.predict(test_X)

In [ ]:
res=[]
for i in range(len(result)):
    res.append(list(result[i]).index(max(result[i])))
function.evaluate(res,test_Y)

In [5]:
import csv
vocab=pd.read_csv('./data/vocab.txt',sep='\t',names=['word','times'],quoting=csv.QUOTE_NONE)
vocab.head()

,word,times
0,，,124634
1,的,94820
2,！,49051
3,？,37853
4,了,33303


In [6]:
def TextCNN(vocab_size,feature_size,embed_size,num_classes,num_filters,filter_sizes,regularizer_lambda,dropout_rate):
    inputs=keras.Input(shape=(feature_size,),name='input_data')
    embed_initer=keras.initializers.RandomUniform(minval=-1,maxval=1)
    embed=keras.layers.Embedding(vocab_size,embed_size,embeddings_initializer=embed_initer,input_length=feature_size,name='embedding')(inputs)
    embed=keras.layers.Reshape((feature_size,embed_size,1),name='add_channel')(embed)
    pool_outputs=[]
    for filter_size in list(map(int,filter_sizes.split(','))):
        filter_shape=(filter_size,embed_size)
        conv=keras.layers.Conv2D(num_filters,filter_shape,strides=(1,1),padding='valid',
                                data_format='channels_last',activation='relu',
                                kernel_initializer='glorot_normal',bias_initializer=keras.initializers.constant(0.1),
                                name='convolution_{:d}'.format(filter_size))(embed)
        max_pool_shape=(feature_size-filter_size+1,1)
        pool=keras.layers.MaxPool2D(pool_size=max_pool_shape,strides=(1,1),padding='valid',data_format='channels_last',name='max_pooling_{:d}'.format(filter_size))(conv)
        pool_outputs.append(pool)
    pool_outputs=keras.layers.concatenate(pool_outputs,axis=-1,name='concatenate')
    pool_outputs=keras.layers.Flatten(data_format='channels_last',name='flatten')(pool_outputs)
    pool_outputs=keras.layers.Dropout(dropout_rate,name='dropout')(pool_outputs)
    outputs=keras.layers.Dense(num_classes,activation='softmax',kernel_initializer='glorot_normal',bias_initializer=keras.initializers.constant(0.1),
                                kernel_regularizer=keras.regularizers.l2(regularizer_lambda),bias_regularizer=keras.regularizers.l2(regularizer_lambda),
                                name='dense')(pool_outputs)
    model=keras.Model(inputs=inputs,outputs=outputs)
    model.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=['accuracy'])
    return model

In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
model=TextCNN(len(vocab),768,512,18,128,'3,4,5',0.01,0.2)

In [8]:
model.fit(train_X,train_Y,epochs=100,batch_size=16,validation_data=(dev_X,dev_Y))

Epoch 1/100


InvalidArgumentError:  indices[13,478] = -1 is not in [0, 150345)
	 [[node model/embedding/embedding_lookup
 (defined at D:\Anaconda\envs\Graduation\lib\site-packages\keras\layers\embeddings.py:191)
]] [Op:__inference_train_function_1035]

Errors may have originated from an input operation.
Input Source operations connected to node model/embedding/embedding_lookup:
In[0] model/embedding/embedding_lookup/699:	
In[1] model/embedding/Cast (defined at D:\Anaconda\envs\Graduation\lib\site-packages\keras\layers\embeddings.py:190)

Operation defined at: (most recent call last)
>>>   File "D:\Anaconda\envs\Graduation\lib\runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "D:\Anaconda\envs\Graduation\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "D:\Anaconda\envs\Graduation\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "D:\Anaconda